## Notebook for CodeSpace

#### Init Git

In [ ]:
## Basic import of modules
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
## data acquisition
## using kaggle dataset
movies_df=pd.read_csv('movies_df.csv')

### Columns to be dropped
* Budget
* Homepage
* Original_language
* Original_title
* Popularity
* Production_companies
* Production_countries
* Release_date (Good if you want to make it in time period)
* Revenue
* Runtime
* Spoken_Language
* Released ??
* Tagline
* Vote_avg
* movie_id

In [ ]:
movies_df.info()

In [ ]:
credit_df=pd.read_csv('tmdb_5000_credits.csv')

## Merge two dataframes

In [ ]:
movies_df=movies_df.merge(credit_df,on='title')